In [113]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import os

In [114]:
os.makedirs('./output_csv', exist_ok=True)
os.makedirs('./input_csv', exist_ok=True)

In [115]:
df_bloomberg = pd.DataFrame()

for year in range(2000, 2026):
    try:
        df_year = pd.read_csv(f"./input_csv/bloomberg_raw/{year}.csv")
        df_bloomberg = pd.concat([df_bloomberg, df_year])
    except FileNotFoundError:
        continue

In [116]:
df_bloomberg['Filing Term Price Range'] = df_bloomberg['Filing Term Price Range'].str.replace('\n', '').str.strip()
df_bloomberg['Trade Date (US)'] = pd.to_datetime(df_bloomberg['Trade Date (US)'], format='%m/%d/%y', errors='coerce')
df_bloomberg['Trade Date (US) FoM'] = df_bloomberg['Trade Date (US)'].dt.to_period('M').dt.to_timestamp()
df_bloomberg = df_bloomberg.dropna(thresh=int(df_bloomberg.shape[1] * 0.8))

In [117]:
df_dff = pd.read_csv('./input_csv/DFF.csv')
df_dff.rename(columns={'observation_date': 'Trade Date (US) FoM'}, inplace=True)
df_dff['Trade Date (US) FoM'] = pd.to_datetime(df_dff['Trade Date (US) FoM'], errors='coerce')


df_cpi = pd.read_csv('./input_csv/CPI.csv')
df_cpi.rename(columns={'observation_date': 'Trade Date (US) FoM'}, inplace=True)
df_cpi['Trade Date (US) FoM'] = pd.to_datetime(df_cpi['Trade Date (US) FoM'], errors='coerce')


df_csent = pd.read_csv('./input_csv/UMCSENT.csv')
df_csent.rename(columns={'observation_date': 'Trade Date (US) FoM'}, inplace=True)
df_csent['Trade Date (US) FoM'] = pd.to_datetime(df_csent['Trade Date (US) FoM'], errors='coerce')


df_unrate = pd.read_csv('./input_csv/UNRATE.csv')
df_unrate.rename(columns={'observation_date': 'Trade Date (US) FoM'}, inplace=True)
df_unrate['Trade Date (US) FoM'] = pd.to_datetime(df_unrate['Trade Date (US) FoM'], errors='coerce')


In [118]:
df_bloomberg = (df_bloomberg.merge(df_dff, 
                                  on='Trade Date (US) FoM', 
                                  how='left')
                            .merge(df_cpi, 
                                  on='Trade Date (US) FoM', 
                                  how='left')
                            .merge(df_csent, 
                                  on='Trade Date (US) FoM', 
                                  how='left')
                            .merge(df_unrate, 
                                  on='Trade Date (US) FoM', 
                                  how='left'))

In [119]:
df_bloomberg.rename(columns={'CPALTT01USM657N': 'CPI',
                              'DFF': 'Fed Rate',
                              'UMCSENT': 'Consumer Confidence',
                              'UNRATE': 'Unemployment Rate'
                             }, inplace=True)

In [120]:
df_bloomberg.drop(columns=['Trade Date (US) FoM'], inplace=True)

In [121]:
df_bloomberg.to_csv('./output_csv/bloomberg_data.csv', index=False)
